Goal is to analyze RNA-seq data to determine whether/how ectopic expression of Airn from Rosa26 in RMCE-Airn cells impacts gene expression (relative to RMCE-empty, also to RMCE-Airn no-dox). May also compare to RMCE-Xist.

For reference, using Jupyter notebook 2022-01-31 RMCE Xist RNA-seq analysis

In [35]:
import os
#show my my working directoty
os.getcwd()

'C:\\Users\\jbt\\OneDrive - University of North Carolina at Chapel Hill\\Postdoc (non-Dropbox)\\Bioinformatics\\2023-04-11_RMCE_Airn_Xist_RNA-seq'

In [36]:
#list what is in my working directory
os.listdir()

['.ipynb_checkpoints',
 '2023-04-11_DESeq2_notes.txt',
 '2023-04-11_RMCE-Airn_Xist_RNA-seq.ipynb',
 '2023-04-11_RMCE_Airn3d_vs_empty_DESeq2_results.csv',
 '2023-04-11_RMCE_Airn7d_vs_empty_DESeq2_results.csv',
 '2023-04-11_RMCE_Airn_Xist_empty_RNAseq_SNP_reads_all_data.txt',
 '2023-04-11_RMCE_Airn_Xist_RNA-seq_processing_notes.txt',
 '2023-04-11_RMCE_B6_reads_Airn3d_empty_for_DESeq2.csv',
 '2023-04-11_RMCE_B6_reads_Airn7d_empty_for_DESeq2.csv',
 '2023-04-11_RMCE_B6_reads_Xist1000ngmL_empty_for_DESeq2.csv',
 '2023-04-11_RMCE_B6_reads_Xist10ngmL_empty_for_DESeq2.csv',
 '2023-04-11_RMCE_Xist1000_vs_empty_DESeq2_results.csv',
 '2023-04-11_RMCE_Xist10_vs_empty_DESeq2_results.csv',
 'Airn3d_empty_MAplot.pdf',
 'Airn3d_empty_MAplot_chr6.pdf',
 'Airn7d_empty_MAplot.pdf',
 'Airn7d_empty_MAplot_chr6.pdf',
 'Xist1000ngmL_empty_MAplot.pdf',
 'Xist1000ngmL_empty_MAplot_chr6.pdf',
 'Xist10ngmL_empty_MAplot.pdf',
 'Xist10ngmL_empty_MAplot_chr6.pdf']

In [37]:
import pandas as pd
#want to be able to see all columns in large dataframes
pd.set_option('display.max_columns', None)

#read in my large data file containing B6/CAST-parsed reads (exon, ejc, intron, total) for each sample
#This text file was copied into my computer's working directory Bioinformatics\\2023-02-06_RMCE-Airn_RNA-seq, shown above
#pd.read_table reads in tab-delimited text files by default
df = pd.read_table('2023-04-11_RMCE_Airn_Xist_empty_RNAseq_SNP_reads_all_data.txt')
df

,id,gene,chr,start,end,strand,Airn2_7d_repA_RNAseq_B6_exon,Airn2_7d_repA_RNAseq_B6_ejc,Airn2_7d_repA_RNAseq_B6_intron,Airn2_7d_repA_RNAseq_B6_total,Airn2_7d_repA_RNAseq_cast_exon,Airn2_7d_repA_RNAseq_cast_ejc,Airn2_7d_repA_RNAseq_cast_intron,Airn2_7d_repA_RNAseq_cast_total,Airn2_7d_repB_RNAseq_B6_exon,Airn2_7d_repB_RNAseq_B6_ejc,Airn2_7d_repB_RNAseq_B6_intron,Airn2_7d_repB_RNAseq_B6_total,Airn2_7d_repB_RNAseq_cast_exon,Airn2_7d_repB_RNAseq_cast_ejc,Airn2_7d_repB_RNAseq_cast_intron,Airn2_7d_repB_RNAseq_cast_total,Airn2_repA_RNAseq_B6_exon,Airn2_repA_RNAseq_B6_ejc,Airn2_repA_RNAseq_B6_intron,Airn2_repA_RNAseq_B6_total,Airn2_repA_RNAseq_cast_exon,Airn2_repA_RNAseq_cast_ejc,Airn2_repA_RNAseq_cast_intron,Airn2_repA_RNAseq_cast_total,Airn2_repB_RNAseq_B6_exon,Airn2_repB_RNAseq_B6_ejc,Airn2_repB_RNAseq_B6_intron,Airn2_repB_RNAseq_B6_total,Airn2_repB_RNAseq_cast_exon,Airn2_repB_RNAseq_cast_ejc,Airn2_repB_RNAseq_cast_intron,Airn2_repB_RNAseq_cast_total,Airn5_7d_repA_RNAseq_B6_exon,Airn5_7d_repA_RNAseq_B6_ejc,Airn5_7d_repA_RNAseq_B6_intron,Airn5_7d_repA_RNAseq_B6_total,Airn5_7d_repA_RNAseq_cast_exon,Airn5_7d_repA_RNAseq_cast_ejc,Airn5_7d_repA_RNAseq_cast_intron,Airn5_7d_repA_RNAseq_cast_total,Airn5_7d_repB_RNAseq_B6_exon,Airn5_7d_repB_RNAseq_B6_ejc,Airn5_7d_repB_RNAseq_B6_intron,Airn5_7d_repB_RNAseq_B6_total,Airn5_7d_repB_RNAseq_cast_exon,Airn5_7d_repB_RNAseq_cast_ejc,Airn5_7d_repB_RNAseq_cast_intron,Airn5_7d_repB_RNAseq_cast_total,Airn5_repA_RNAseq_B6_exon,Airn5_repA_RNAseq_B6_ejc,Airn5_repA_RNAseq_B6_intron,Airn5_repA_RNAseq_B6_total,Airn5_repA_RNAseq_cast_exon,Airn5_repA_RNAseq_cast_ejc,Airn5_repA_RNAseq_cast_intron,Airn5_repA_RNAseq_cast_total,Airn5_repB_RNAseq_B6_exon,Airn5_repB_RNAseq_B6_ejc,Airn5_repB_RNAseq_B6_intron,Airn5_repB_RNAseq_B6_total,Airn5_repB_RNAseq_cast_exon,Airn5_repB_RNAseq_cast_ejc,Airn5_repB_RNAseq_cast_intron,Airn5_repB_RNAseq_cast_total,empty1_RNAseq_B6_exon,empty1_RNAseq_B6_ejc,empty1_RNAseq_B6_intron,empty1_RNAseq_B6_total,empty1_RNAseq_cast_exon,empty1_RNAseq_cast_ejc,empty1_RNAseq_cast_intron,empty1_RNAseq_cast_total,empty2_RNAseq_B6_exon,empty2_RNAseq_B6_ejc,empty2_RNAseq_B6_intron,empty2_RNAseq_B6_total,empty2_RNAseq_cast_exon,empty2_RNAseq_cast_ejc,empty2_RNAseq_cast_intron,empty2_RNAseq_cast_total,empty3_RNAseq_B6_exon,empty3_RNAseq_B6_ejc,empty3_RNAseq_B6_intron,empty3_RNAseq_B6_total,empty3_RNAseq_cast_exon,empty3_RNAseq_cast_ejc,empty3_RNAseq_cast_intron,empty3_RNAseq_cast_total,empty4_RNAseq_B6_exon,empty4_RNAseq_B6_ejc,empty4_RNAseq_B6_intron,empty4_RNAseq_B6_total,empty4_RNAseq_cast_exon,empty4_RNAseq_cast_ejc,empty4_RNAseq_cast_intron,empty4_RNAseq_cast_total,Xist1_RNAseq_B6_exon,Xist1_RNAseq_B6_ejc,Xist1_RNAseq_B6_intron,Xist1_RNAseq_B6_total,Xist1_RNAseq_cast_exon,Xist1_RNAseq_cast_ejc,Xist1_RNAseq_cast_intron,Xist1_RNAseq_cast_total,Xist3_10ngmL_repA_RNAseq_B6_exon,Xist3_10ngmL_repA_RNAseq_B6_ejc,Xist3_10ngmL_repA_RNAseq_B6_intron,Xist3_10ngmL_repA_RNAseq_B6_total,Xist3_10ngmL_repA_RNAseq_cast_exon,Xist3_10ngmL_repA_RNAseq_cast_ejc,Xist3_10ngmL_repA_RNAseq_cast_intron,Xist3_10ngmL_repA_RNAseq_cast_total,Xist3_10ngmL_repB_RNAseq_B6_exon,Xist3_10ngmL_repB_RNAseq_B6_ejc,Xist3_10ngmL_repB_RNAseq_B6_intron,Xist3_10ngmL_repB_RNAseq_B6_total,Xist3_10ngmL_repB_RNAseq_cast_exon,Xist3_10ngmL_repB_RNAseq_cast_ejc,Xist3_10ngmL_repB_RNAseq_cast_intron,Xist3_10ngmL_repB_RNAseq_cast_total,Xist3_RNAseq_B6_exon,Xist3_RNAseq_B6_ejc,Xist3_RNAseq_B6_intron,Xist3_RNAseq_B6_total,Xist3_RNAseq_cast_exon,Xist3_RNAseq_cast_ejc,Xist3_RNAseq_cast_intron,Xist3_RNAseq_cast_total,Xist4_RNAseq_B6_exon,Xist4_RNAseq_B6_ejc,Xist4_RNAseq_B6_intron,Xist4_RNAseq_B6_total,Xist4_RNAseq_cast_exon,Xist4_RNAseq_cast_ejc,Xist4_RNAseq_cast_intron,Xist4_RNAseq_cast_total,Xist5_10ngmL_repA_RNAseq_B6_exon,Xist5_10ngmL_repA_RNAseq_B6_ejc,Xist5_10ngmL_repA_RNAseq_B6_intron,Xist5_10ngmL_repA_RNAseq_B6_total,Xist5_10ngmL_repA_RNAseq_cast_exon,Xist5_10ngmL_repA_RNAseq_cast_ejc,Xist5_10ngmL_repA_RNAseq_cast_intron,Xist5_10ngmL_repA_RNAseq_cast_

In [38]:
#For my first analysis, I want to subset this dataframe down to only the "B6_total" columns
#make dataframe containing ther first 6 "identifier" columns - specify start (inclusive) of 0, end (NOT INCLUSIVE!) of 6
df_ID_columns = df.iloc[:,0:6]
df_ID_columns

,id,gene,chr,start,end,strand
0,4933401J01Rik_chr1_3073253_3074322_(+),4933401J01Rik,chr1,3073253,3074322,+
1,Gm26206_chr1_3102016_3102125_(+),Gm26206,chr1,3102016,3102125,+
2,Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-
3,Gm18956_chr1_3252757_3253236_(+),Gm18956,chr1,3252757,3253236,+
4,Gm37180_chr1_3365731_3368549_(-),Gm37180,chr1,3365731,3368549,-
...,...,...,...,...,...,...
55396,Gm21860_chrY_90752427_90755467_(-),Gm21860,chrY,90752427,90755467,-
55397,Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+
55398,Gm47283_chrY_90784738_90816465_(+),Gm47283,chrY,90784738,90816465,+
55399,Gm21742_chrY_90837413_90844040_(+),Gm21742,chrY,90837413,90844040,+


In [39]:
#make dataframe containing only the "B6_total" columns
df_B6_total_data = df.loc[:, df.columns.str.contains('B6_total')]
df_B6_total_data

,Airn2_7d_repA_RNAseq_B6_total,Airn2_7d_repB_RNAseq_B6_total,Airn2_repA_RNAseq_B6_total,Airn2_repB_RNAseq_B6_total,Airn5_7d_repA_RNAseq_B6_total,Airn5_7d_repB_RNAseq_B6_total,Airn5_repA_RNAseq_B6_total,Airn5_repB_RNAseq_B6_total,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist1_RNAseq_B6_total,Xist3_10ngmL_repA_RNAseq_B6_total,Xist3_10ngmL_repB_RNAseq_B6_total,Xist3_RNAseq_B6_total,Xist4_RNAseq_B6_total,Xist5_10ngmL_repA_RNAseq_B6_total,Xist5_10ngmL_repB_RNAseq_B6_total,Xist5_RNAseq_B6_total
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,276,272,298,268,80,102,107,116,313,255,374,40,197,158,400,235,250,126,338,262
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,2,1,0,2,0,0,2,1,0,0,3,2,1,2,0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55396,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55397,21,19,16,27,10,7,7,9,31,11,16,17,15,11,15,10,12,3,15,15
55398,8277,8488,8400,7370,7863,8571,9659,7967,12349,9059,7618,8334,5441,5525,8687,8327,8477,6020,11111,9342
55399,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
#Now combine df_ID_columns and df_B6_total_data

df_B6_total = pd.concat([df_ID_columns, df_B6_total_data], axis=1)
df_B6_total

,id,gene,chr,start,end,strand,Airn2_7d_repA_RNAseq_B6_total,Airn2_7d_repB_RNAseq_B6_total,Airn2_repA_RNAseq_B6_total,Airn2_repB_RNAseq_B6_total,Airn5_7d_repA_RNAseq_B6_total,Airn5_7d_repB_RNAseq_B6_total,Airn5_repA_RNAseq_B6_total,Airn5_repB_RNAseq_B6_total,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist1_RNAseq_B6_total,Xist3_10ngmL_repA_RNAseq_B6_total,Xist3_10ngmL_repB_RNAseq_B6_total,Xist3_RNAseq_B6_total,Xist4_RNAseq_B6_total,Xist5_10ngmL_repA_RNAseq_B6_total,Xist5_10ngmL_repB_RNAseq_B6_total,Xist5_RNAseq_B6_total
0,4933401J01Rik_chr1_3073253_3074322_(+),4933401J01Rik,chr1,3073253,3074322,+,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Gm26206_chr1_3102016_3102125_(+),Gm26206,chr1,3102016,3102125,+,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,276,272,298,268,80,102,107,116,313,255,374,40,197,158,400,235,250,126,338,262
3,Gm18956_chr1_3252757_3253236_(+),Gm18956,chr1,3252757,3253236,+,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Gm37180_chr1_3365731_3368549_(-),Gm37180,chr1,3365731,3368549,-,1,1,2,1,0,2,0,0,2,1,0,0,3,2,1,2,0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55396,Gm21860_chrY_90752427_90755467_(-),Gm21860,chrY,90752427,90755467,-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55397,Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,21,19,16,27,10,7,7,9,31,11,16,17,15,11,15,10,12,3,15,15
55398,Gm47283_chrY_90784738_90816465_(+),Gm47283,chrY,90784738,90816465,+,8277,8488,8400,7370,7863,8571,9659,7967,12349,9059,7618,8334,5441,5525,8687,8327,8477,6020,11111,9342
55399,Gm21742_chrY_90837413_90844040_(+),Gm21742,chrY,90837413,90844040,+,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


I want to run DESeq2 (R package) to identify differentially expressed genes between the following groups of samples:

1. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Xist (4 clones, 1 sample each)
2. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Xist 10 ng/mL (2 clones, 2 samples each)
3. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Airn (2 clones, 2 samples each)
4. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Airn 7 d (2 clones, 2 samples each)

For DESeq comparison #1, further subset df_B6_total to contain only these 8 samples' data columns.

In [41]:
df_B6_total_empty_data = df_B6_total.loc[:, df_B6_total.columns.str.contains('empty')]
df_B6_total_Xist1000dox_data = df_B6_total[['Xist1_RNAseq_B6_total', 'Xist3_RNAseq_B6_total', 'Xist4_RNAseq_B6_total', 'Xist5_RNAseq_B6_total',]]

df_B6_total_empty_Xist1000dox = pd.concat([df_ID_columns, df_B6_total_empty_data, df_B6_total_Xist1000dox_data,], axis=1)
df_B6_total_empty_Xist1000dox

,id,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist1_RNAseq_B6_total,Xist3_RNAseq_B6_total,Xist4_RNAseq_B6_total,Xist5_RNAseq_B6_total
0,4933401J01Rik_chr1_3073253_3074322_(+),4933401J01Rik,chr1,3073253,3074322,+,0,0,0,0,0,0,0,0
1,Gm26206_chr1_3102016_3102125_(+),Gm26206,chr1,3102016,3102125,+,0,0,0,0,0,0,0,0
2,Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,197,235,250,262
3,Gm18956_chr1_3252757_3253236_(+),Gm18956,chr1,3252757,3253236,+,0,0,0,0,0,0,0,0
4,Gm37180_chr1_3365731_3368549_(-),Gm37180,chr1,3365731,3368549,-,2,1,0,0,3,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55396,Gm21860_chrY_90752427_90755467_(-),Gm21860,chrY,90752427,90755467,-,0,0,0,0,0,0,0,0
55397,Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,15,10,12,15
55398,Gm47283_chrY_90784738_90816465_(+),Gm47283,chrY,90784738,90816465,+,12349,9059,7618,8334,5441,8327,8477,9342
55399,Gm21742_chrY_90837413_90844040_(+),Gm21742,chrY,90837413,90844040,+,0,0,0,0,0,0,0,0


Last step before running DESeq2: pre-filter the genes to include only those that have an average of 5 reads per sample (so a total of 40 reads across these 8 samples).

In [42]:
df_B6_total_empty_Xist1000dox_filtered = df_B6_total_empty_Xist1000dox[df_B6_total_empty_Xist1000dox.loc[:, "empty1_RNAseq_B6_total" : "Xist5_RNAseq_B6_total"].sum(axis=1) >= 40]
df_B6_total_empty_Xist1000dox_filtered

,id,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist1_RNAseq_B6_total,Xist3_RNAseq_B6_total,Xist4_RNAseq_B6_total,Xist5_RNAseq_B6_total
2,Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,197,235,250,262
31,Mrpl15_chr1_4773206_4785739_(-),Mrpl15,chr1,4773206,4785739,-,440,311,251,257,237,351,186,310
33,Lypla1_chr1_4807788_4848410_(+),Lypla1,chr1,4807788,4848410,+,513,249,272,242,241,360,216,325
34,Gm37988_chr1_4807892_4886770_(+),Gm37988,chr1,4807892,4886770,+,369,247,211,241,230,263,176,215
35,Tcea1_chr1_4857814_4897909_(+),Tcea1,chr1,4857814,4897909,+,316,155,171,159,160,192,121,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53819,Gm15246_chrX_169320622_169368190_(+),Gm15246,chrX,169320622,169368190,+,99,72,60,72,81,78,76,82
53820,BC022960_chrX_169368189_169370003_(+),BC022960,chrX,169368189,169370003,+,8,5,6,9,6,8,17,8
53822,Mid1_chrX_169685199_170005736_(+),Mid1,chrX,169685199,170005736,+,23106,647,662,570,379,579,472,594
55397,Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,15,10,12,15


For DESeq2, will need this sata in the form of a matrix. Matrices in R can have row names and column names - will want a single rowname, so I will use the "id" column here:

In [43]:
df_B6_total_empty_Xist1000dox_filtered2 = df_B6_total_empty_Xist1000dox_filtered.set_index('id')
df_B6_total_empty_Xist1000dox_filtered2

,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist1_RNAseq_B6_total,Xist3_RNAseq_B6_total,Xist4_RNAseq_B6_total,Xist5_RNAseq_B6_total
id,,,,,,,,,,,,,
Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,197,235,250,262
Mrpl15_chr1_4773206_4785739_(-),Mrpl15,chr1,4773206,4785739,-,440,311,251,257,237,351,186,310
Lypla1_chr1_4807788_4848410_(+),Lypla1,chr1,4807788,4848410,+,513,249,272,242,241,360,216,325
Gm37988_chr1_4807892_4886770_(+),Gm37988,chr1,4807892,4886770,+,369,247,211,241,230,263,176,215
Tcea1_chr1_4857814_4897909_(+),Tcea1,chr1,4857814,4897909,+,316,155,171,159,160,192,121,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gm15246_chrX_169320622_169368190_(+),Gm15246,chrX,169320622,169368190,+,99,72,60,72,81,78,76,82
BC022960_chrX_169368189_169370003_(+),BC022960,chrX,169368189,169370003,+,8,5,6,9,6,8,17,8
Mid1_chrX_169685199_170005736_(+),Mid1,chrX,169685199,170005736,+,23106,647,662,570,379,579,472,594


In [44]:
df_B6_total_empty_Xist1000dox_filtered2.index.names = [None]
df_B6_total_empty_Xist1000dox_filtered2

,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist1_RNAseq_B6_total,Xist3_RNAseq_B6_total,Xist4_RNAseq_B6_total,Xist5_RNAseq_B6_total
Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,197,235,250,262
Mrpl15_chr1_4773206_4785739_(-),Mrpl15,chr1,4773206,4785739,-,440,311,251,257,237,351,186,310
Lypla1_chr1_4807788_4848410_(+),Lypla1,chr1,4807788,4848410,+,513,249,272,242,241,360,216,325
Gm37988_chr1_4807892_4886770_(+),Gm37988,chr1,4807892,4886770,+,369,247,211,241,230,263,176,215
Tcea1_chr1_4857814_4897909_(+),Tcea1,chr1,4857814,4897909,+,316,155,171,159,160,192,121,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gm15246_chrX_169320622_169368190_(+),Gm15246,chrX,169320622,169368190,+,99,72,60,72,81,78,76,82
BC022960_chrX_169368189_169370003_(+),BC022960,chrX,169368189,169370003,+,8,5,6,9,6,8,17,8
Mid1_chrX_169685199_170005736_(+),Mid1,chrX,169685199,170005736,+,23106,647,662,570,379,579,472,594
Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,15,10,12,15


In [45]:
#Now remove the ID/metadata columns
df_B6_total_empty_Xist1000dox_filtered3 = df_B6_total_empty_Xist1000dox_filtered2.iloc[:, 5:14]
df_B6_total_empty_Xist1000dox_filtered3

,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist1_RNAseq_B6_total,Xist3_RNAseq_B6_total,Xist4_RNAseq_B6_total,Xist5_RNAseq_B6_total
Xkr4_chr1_3205901_3671498_(-),313,255,374,40,197,235,250,262
Mrpl15_chr1_4773206_4785739_(-),440,311,251,257,237,351,186,310
Lypla1_chr1_4807788_4848410_(+),513,249,272,242,241,360,216,325
Gm37988_chr1_4807892_4886770_(+),369,247,211,241,230,263,176,215
Tcea1_chr1_4857814_4897909_(+),316,155,171,159,160,192,121,157
...,...,...,...,...,...,...,...,...
Gm15246_chrX_169320622_169368190_(+),99,72,60,72,81,78,76,82
BC022960_chrX_169368189_169370003_(+),8,5,6,9,6,8,17,8
Mid1_chrX_169685199_170005736_(+),23106,647,662,570,379,579,472,594
Mid1-ps1_chrY_90753057_90763485_(+),31,11,16,17,15,10,12,15


In [46]:
df_B6_total_empty_Xist1000dox_filtered3.to_csv('2023-04-11_RMCE_B6_reads_Xist1000ngmL_empty_for_DESeq2.csv')

In [47]:
df_B6_total[df_B6_total.gene == 'Xist']

,id,gene,chr,start,end,strand,Airn2_7d_repA_RNAseq_B6_total,Airn2_7d_repB_RNAseq_B6_total,Airn2_repA_RNAseq_B6_total,Airn2_repB_RNAseq_B6_total,Airn5_7d_repA_RNAseq_B6_total,Airn5_7d_repB_RNAseq_B6_total,Airn5_repA_RNAseq_B6_total,Airn5_repB_RNAseq_B6_total,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist1_RNAseq_B6_total,Xist3_10ngmL_repA_RNAseq_B6_total,Xist3_10ngmL_repB_RNAseq_B6_total,Xist3_RNAseq_B6_total,Xist4_RNAseq_B6_total,Xist5_10ngmL_repA_RNAseq_B6_total,Xist5_10ngmL_repB_RNAseq_B6_total,Xist5_RNAseq_B6_total
52813,Xist_chrX_103460366_103483254_(-),Xist,chrX,103460366,103483254,-,16,25,19,19,15,35,21,15,3,15,15,14,19283,2686,1731,17770,21382,2138,2159,16099


I want to run DESeq2 (R package) to identify differentially expressed genes between the following groups of samples:

1. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Xist (4 clones, 1 sample each)
2. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Xist 10 ng/mL (2 clones, 2 samples each)
3. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Airn (2 clones, 2 samples each)
4. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Airn 7 d (2 clones, 2 samples each)

For DESeq comparison #2, further subset df_B6_total to contain only these 8 samples' data columns.

In [48]:
df_B6_total_empty_data = df_B6_total.loc[:, df_B6_total.columns.str.contains('empty')]
df_B6_total_Xist10dox_data = df_B6_total[['Xist3_10ngmL_repA_RNAseq_B6_total', 'Xist3_10ngmL_repB_RNAseq_B6_total', 'Xist5_10ngmL_repA_RNAseq_B6_total', 'Xist5_10ngmL_repB_RNAseq_B6_total']]

df_B6_total_empty_Xist10dox = pd.concat([df_ID_columns, df_B6_total_empty_data, df_B6_total_Xist10dox_data,], axis=1)
df_B6_total_empty_Xist10dox

,id,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist3_10ngmL_repA_RNAseq_B6_total,Xist3_10ngmL_repB_RNAseq_B6_total,Xist5_10ngmL_repA_RNAseq_B6_total,Xist5_10ngmL_repB_RNAseq_B6_total
0,4933401J01Rik_chr1_3073253_3074322_(+),4933401J01Rik,chr1,3073253,3074322,+,0,0,0,0,0,0,0,0
1,Gm26206_chr1_3102016_3102125_(+),Gm26206,chr1,3102016,3102125,+,0,0,0,0,0,0,0,0
2,Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,158,400,126,338
3,Gm18956_chr1_3252757_3253236_(+),Gm18956,chr1,3252757,3253236,+,0,0,0,0,0,0,0,0
4,Gm37180_chr1_3365731_3368549_(-),Gm37180,chr1,3365731,3368549,-,2,1,0,0,2,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55396,Gm21860_chrY_90752427_90755467_(-),Gm21860,chrY,90752427,90755467,-,0,0,0,0,0,0,0,0
55397,Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,11,15,3,15
55398,Gm47283_chrY_90784738_90816465_(+),Gm47283,chrY,90784738,90816465,+,12349,9059,7618,8334,5525,8687,6020,11111
55399,Gm21742_chrY_90837413_90844040_(+),Gm21742,chrY,90837413,90844040,+,0,0,0,0,0,0,0,0


Last step before running DESeq2: pre-filter the genes to include only those that have an average of 5 reads per sample (so a total of 40 reads across these 8 samples).

In [49]:
df_B6_total_empty_Xist10dox_filtered = df_B6_total_empty_Xist10dox[df_B6_total_empty_Xist10dox.loc[:, "empty1_RNAseq_B6_total" : "Xist5_10ngmL_repB_RNAseq_B6_total"].sum(axis=1) >= 40]
df_B6_total_empty_Xist10dox_filtered

,id,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist3_10ngmL_repA_RNAseq_B6_total,Xist3_10ngmL_repB_RNAseq_B6_total,Xist5_10ngmL_repA_RNAseq_B6_total,Xist5_10ngmL_repB_RNAseq_B6_total
2,Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,158,400,126,338
11,Gm19938_chr1_3647309_3658904_(-),Gm19938,chr1,3647309,3658904,-,6,3,8,0,2,7,5,10
31,Mrpl15_chr1_4773206_4785739_(-),Mrpl15,chr1,4773206,4785739,-,440,311,251,257,195,287,122,247
33,Lypla1_chr1_4807788_4848410_(+),Lypla1,chr1,4807788,4848410,+,513,249,272,242,183,263,129,204
34,Gm37988_chr1_4807892_4886770_(+),Gm37988,chr1,4807892,4886770,+,369,247,211,241,147,239,130,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53819,Gm15246_chrX_169320622_169368190_(+),Gm15246,chrX,169320622,169368190,+,99,72,60,72,45,74,29,74
53820,BC022960_chrX_169368189_169370003_(+),BC022960,chrX,169368189,169370003,+,8,5,6,9,8,8,6,11
53822,Mid1_chrX_169685199_170005736_(+),Mid1,chrX,169685199,170005736,+,23106,647,662,570,269,502,241,467
55397,Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,11,15,3,15


For DESeq2, will need this sata in the form of a matrix. Matrices in R can have row names and column names - will want a single rowname, so I will use the "id" column here:

In [50]:
df_B6_total_empty_Xist10dox_filtered2 = df_B6_total_empty_Xist10dox_filtered.set_index('id')
df_B6_total_empty_Xist10dox_filtered2

,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist3_10ngmL_repA_RNAseq_B6_total,Xist3_10ngmL_repB_RNAseq_B6_total,Xist5_10ngmL_repA_RNAseq_B6_total,Xist5_10ngmL_repB_RNAseq_B6_total
id,,,,,,,,,,,,,
Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,158,400,126,338
Gm19938_chr1_3647309_3658904_(-),Gm19938,chr1,3647309,3658904,-,6,3,8,0,2,7,5,10
Mrpl15_chr1_4773206_4785739_(-),Mrpl15,chr1,4773206,4785739,-,440,311,251,257,195,287,122,247
Lypla1_chr1_4807788_4848410_(+),Lypla1,chr1,4807788,4848410,+,513,249,272,242,183,263,129,204
Gm37988_chr1_4807892_4886770_(+),Gm37988,chr1,4807892,4886770,+,369,247,211,241,147,239,130,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gm15246_chrX_169320622_169368190_(+),Gm15246,chrX,169320622,169368190,+,99,72,60,72,45,74,29,74
BC022960_chrX_169368189_169370003_(+),BC022960,chrX,169368189,169370003,+,8,5,6,9,8,8,6,11
Mid1_chrX_169685199_170005736_(+),Mid1,chrX,169685199,170005736,+,23106,647,662,570,269,502,241,467


In [51]:
df_B6_total_empty_Xist10dox_filtered2.index.names = [None]
df_B6_total_empty_Xist10dox_filtered2

,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist3_10ngmL_repA_RNAseq_B6_total,Xist3_10ngmL_repB_RNAseq_B6_total,Xist5_10ngmL_repA_RNAseq_B6_total,Xist5_10ngmL_repB_RNAseq_B6_total
Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,158,400,126,338
Gm19938_chr1_3647309_3658904_(-),Gm19938,chr1,3647309,3658904,-,6,3,8,0,2,7,5,10
Mrpl15_chr1_4773206_4785739_(-),Mrpl15,chr1,4773206,4785739,-,440,311,251,257,195,287,122,247
Lypla1_chr1_4807788_4848410_(+),Lypla1,chr1,4807788,4848410,+,513,249,272,242,183,263,129,204
Gm37988_chr1_4807892_4886770_(+),Gm37988,chr1,4807892,4886770,+,369,247,211,241,147,239,130,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gm15246_chrX_169320622_169368190_(+),Gm15246,chrX,169320622,169368190,+,99,72,60,72,45,74,29,74
BC022960_chrX_169368189_169370003_(+),BC022960,chrX,169368189,169370003,+,8,5,6,9,8,8,6,11
Mid1_chrX_169685199_170005736_(+),Mid1,chrX,169685199,170005736,+,23106,647,662,570,269,502,241,467
Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,11,15,3,15


In [52]:
#Now remove the ID/metadata columns
df_B6_total_empty_Xist10dox_filtered3 = df_B6_total_empty_Xist10dox_filtered2.iloc[:, 5:14]
df_B6_total_empty_Xist10dox_filtered3

,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist3_10ngmL_repA_RNAseq_B6_total,Xist3_10ngmL_repB_RNAseq_B6_total,Xist5_10ngmL_repA_RNAseq_B6_total,Xist5_10ngmL_repB_RNAseq_B6_total
Xkr4_chr1_3205901_3671498_(-),313,255,374,40,158,400,126,338
Gm19938_chr1_3647309_3658904_(-),6,3,8,0,2,7,5,10
Mrpl15_chr1_4773206_4785739_(-),440,311,251,257,195,287,122,247
Lypla1_chr1_4807788_4848410_(+),513,249,272,242,183,263,129,204
Gm37988_chr1_4807892_4886770_(+),369,247,211,241,147,239,130,224
...,...,...,...,...,...,...,...,...
Gm15246_chrX_169320622_169368190_(+),99,72,60,72,45,74,29,74
BC022960_chrX_169368189_169370003_(+),8,5,6,9,8,8,6,11
Mid1_chrX_169685199_170005736_(+),23106,647,662,570,269,502,241,467
Mid1-ps1_chrY_90753057_90763485_(+),31,11,16,17,11,15,3,15


In [53]:
df_B6_total_empty_Xist10dox_filtered3.to_csv('2023-04-11_RMCE_B6_reads_Xist10ngmL_empty_for_DESeq2.csv')

I want to run DESeq2 (R package) to identify differentially expressed genes between the following groups of samples:

1. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Xist (4 clones, 1 sample each)
2. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Xist 10 ng/mL (2 clones, 2 samples each)
3. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Airn (2 clones, 2 samples each)
4. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Airn 7 d (2 clones, 2 samples each)

For DESeq comparison #3, further subset df_B6_total to contain only these 8 samples' data columns.

In [54]:
df_B6_total_empty_data = df_B6_total.loc[:, df_B6_total.columns.str.contains('empty')]
df_B6_total_Airn3d_data = df_B6_total[['Airn2_repA_RNAseq_B6_total', 'Airn2_repB_RNAseq_B6_total', 'Airn5_repA_RNAseq_B6_total', 'Airn5_repB_RNAseq_B6_total']]

df_B6_total_empty_Airn3d = pd.concat([df_ID_columns, df_B6_total_empty_data, df_B6_total_Airn3d_data,], axis=1)
df_B6_total_empty_Airn3d

,id,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Airn2_repA_RNAseq_B6_total,Airn2_repB_RNAseq_B6_total,Airn5_repA_RNAseq_B6_total,Airn5_repB_RNAseq_B6_total
0,4933401J01Rik_chr1_3073253_3074322_(+),4933401J01Rik,chr1,3073253,3074322,+,0,0,0,0,0,0,0,0
1,Gm26206_chr1_3102016_3102125_(+),Gm26206,chr1,3102016,3102125,+,0,0,0,0,0,0,0,0
2,Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,298,268,107,116
3,Gm18956_chr1_3252757_3253236_(+),Gm18956,chr1,3252757,3253236,+,0,0,0,0,0,0,0,0
4,Gm37180_chr1_3365731_3368549_(-),Gm37180,chr1,3365731,3368549,-,2,1,0,0,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55396,Gm21860_chrY_90752427_90755467_(-),Gm21860,chrY,90752427,90755467,-,0,0,0,0,0,0,0,0
55397,Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,16,27,7,9
55398,Gm47283_chrY_90784738_90816465_(+),Gm47283,chrY,90784738,90816465,+,12349,9059,7618,8334,8400,7370,9659,7967
55399,Gm21742_chrY_90837413_90844040_(+),Gm21742,chrY,90837413,90844040,+,0,0,0,0,0,0,0,0


Last step before running DESeq2: pre-filter the genes to include only those that have an average of 5 reads per sample (so a total of 40 reads across these 8 samples).

In [55]:
df_B6_total_empty_Airn3d_filtered = df_B6_total_empty_Airn3d[df_B6_total_empty_Airn3d.loc[:, "empty1_RNAseq_B6_total" : "Airn5_repB_RNAseq_B6_total"].sum(axis=1) >= 40]
df_B6_total_empty_Airn3d_filtered

,id,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Airn2_repA_RNAseq_B6_total,Airn2_repB_RNAseq_B6_total,Airn5_repA_RNAseq_B6_total,Airn5_repB_RNAseq_B6_total
2,Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,298,268,107,116
31,Mrpl15_chr1_4773206_4785739_(-),Mrpl15,chr1,4773206,4785739,-,440,311,251,257,298,238,288,203
33,Lypla1_chr1_4807788_4848410_(+),Lypla1,chr1,4807788,4848410,+,513,249,272,242,241,190,348,218
34,Gm37988_chr1_4807892_4886770_(+),Gm37988,chr1,4807892,4886770,+,369,247,211,241,242,196,280,192
35,Tcea1_chr1_4857814_4897909_(+),Tcea1,chr1,4857814,4897909,+,316,155,171,159,165,153,189,143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53819,Gm15246_chrX_169320622_169368190_(+),Gm15246,chrX,169320622,169368190,+,99,72,60,72,92,71,126,86
53820,BC022960_chrX_169368189_169370003_(+),BC022960,chrX,169368189,169370003,+,8,5,6,9,5,1,20,9
53822,Mid1_chrX_169685199_170005736_(+),Mid1,chrX,169685199,170005736,+,23106,647,662,570,532,429,312,257
55397,Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,16,27,7,9


For DESeq2, will need this sata in the form of a matrix. Matrices in R can have row names and column names - will want a single rowname, so I will use the "id" column here:

In [56]:
df_B6_total_empty_Airn3d_filtered2 = df_B6_total_empty_Airn3d_filtered.set_index('id')
df_B6_total_empty_Airn3d_filtered2

,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Airn2_repA_RNAseq_B6_total,Airn2_repB_RNAseq_B6_total,Airn5_repA_RNAseq_B6_total,Airn5_repB_RNAseq_B6_total
id,,,,,,,,,,,,,
Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,298,268,107,116
Mrpl15_chr1_4773206_4785739_(-),Mrpl15,chr1,4773206,4785739,-,440,311,251,257,298,238,288,203
Lypla1_chr1_4807788_4848410_(+),Lypla1,chr1,4807788,4848410,+,513,249,272,242,241,190,348,218
Gm37988_chr1_4807892_4886770_(+),Gm37988,chr1,4807892,4886770,+,369,247,211,241,242,196,280,192
Tcea1_chr1_4857814_4897909_(+),Tcea1,chr1,4857814,4897909,+,316,155,171,159,165,153,189,143
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gm15246_chrX_169320622_169368190_(+),Gm15246,chrX,169320622,169368190,+,99,72,60,72,92,71,126,86
BC022960_chrX_169368189_169370003_(+),BC022960,chrX,169368189,169370003,+,8,5,6,9,5,1,20,9
Mid1_chrX_169685199_170005736_(+),Mid1,chrX,169685199,170005736,+,23106,647,662,570,532,429,312,257


In [57]:
df_B6_total_empty_Airn3d_filtered2.index.names = [None]
df_B6_total_empty_Airn3d_filtered2

,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Airn2_repA_RNAseq_B6_total,Airn2_repB_RNAseq_B6_total,Airn5_repA_RNAseq_B6_total,Airn5_repB_RNAseq_B6_total
Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,298,268,107,116
Mrpl15_chr1_4773206_4785739_(-),Mrpl15,chr1,4773206,4785739,-,440,311,251,257,298,238,288,203
Lypla1_chr1_4807788_4848410_(+),Lypla1,chr1,4807788,4848410,+,513,249,272,242,241,190,348,218
Gm37988_chr1_4807892_4886770_(+),Gm37988,chr1,4807892,4886770,+,369,247,211,241,242,196,280,192
Tcea1_chr1_4857814_4897909_(+),Tcea1,chr1,4857814,4897909,+,316,155,171,159,165,153,189,143
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gm15246_chrX_169320622_169368190_(+),Gm15246,chrX,169320622,169368190,+,99,72,60,72,92,71,126,86
BC022960_chrX_169368189_169370003_(+),BC022960,chrX,169368189,169370003,+,8,5,6,9,5,1,20,9
Mid1_chrX_169685199_170005736_(+),Mid1,chrX,169685199,170005736,+,23106,647,662,570,532,429,312,257
Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,16,27,7,9


In [58]:
#Now remove the ID/metadata columns
df_B6_total_empty_Airn3d_filtered3 = df_B6_total_empty_Airn3d_filtered2.iloc[:, 5:14]
df_B6_total_empty_Airn3d_filtered3

,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Airn2_repA_RNAseq_B6_total,Airn2_repB_RNAseq_B6_total,Airn5_repA_RNAseq_B6_total,Airn5_repB_RNAseq_B6_total
Xkr4_chr1_3205901_3671498_(-),313,255,374,40,298,268,107,116
Mrpl15_chr1_4773206_4785739_(-),440,311,251,257,298,238,288,203
Lypla1_chr1_4807788_4848410_(+),513,249,272,242,241,190,348,218
Gm37988_chr1_4807892_4886770_(+),369,247,211,241,242,196,280,192
Tcea1_chr1_4857814_4897909_(+),316,155,171,159,165,153,189,143
...,...,...,...,...,...,...,...,...
Gm15246_chrX_169320622_169368190_(+),99,72,60,72,92,71,126,86
BC022960_chrX_169368189_169370003_(+),8,5,6,9,5,1,20,9
Mid1_chrX_169685199_170005736_(+),23106,647,662,570,532,429,312,257
Mid1-ps1_chrY_90753057_90763485_(+),31,11,16,17,16,27,7,9


In [59]:
df_B6_total_empty_Airn3d_filtered3.to_csv('2023-04-11_RMCE_B6_reads_Airn3d_empty_for_DESeq2.csv')

I want to run DESeq2 (R package) to identify differentially expressed genes between the following groups of samples:

1. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Xist (4 clones, 1 sample each)
2. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Xist 10 ng/mL (2 clones, 2 samples each)
3. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Airn (2 clones, 2 samples each)
4. RMCE-empty (4 clones, 1 sample each) vs. RMCE-Airn 7 d (2 clones, 2 samples each)

For DESeq comparison #4, further subset df_B6_total to contain only these 8 samples' data columns.

In [60]:
df_B6_total_empty_data = df_B6_total.loc[:, df_B6_total.columns.str.contains('empty')]
df_B6_total_Airn7d_data = df_B6_total.loc[:, df_B6_total.columns.str.contains('7d')]

df_B6_total_empty_Airn7d = pd.concat([df_ID_columns, df_B6_total_empty_data, df_B6_total_Airn7d_data,], axis=1)
df_B6_total_empty_Airn7d

,id,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Airn2_7d_repA_RNAseq_B6_total,Airn2_7d_repB_RNAseq_B6_total,Airn5_7d_repA_RNAseq_B6_total,Airn5_7d_repB_RNAseq_B6_total
0,4933401J01Rik_chr1_3073253_3074322_(+),4933401J01Rik,chr1,3073253,3074322,+,0,0,0,0,0,0,0,0
1,Gm26206_chr1_3102016_3102125_(+),Gm26206,chr1,3102016,3102125,+,0,0,0,0,0,0,0,0
2,Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,276,272,80,102
3,Gm18956_chr1_3252757_3253236_(+),Gm18956,chr1,3252757,3253236,+,0,0,0,0,0,0,0,0
4,Gm37180_chr1_3365731_3368549_(-),Gm37180,chr1,3365731,3368549,-,2,1,0,0,1,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55396,Gm21860_chrY_90752427_90755467_(-),Gm21860,chrY,90752427,90755467,-,0,0,0,0,0,0,0,0
55397,Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,21,19,10,7
55398,Gm47283_chrY_90784738_90816465_(+),Gm47283,chrY,90784738,90816465,+,12349,9059,7618,8334,8277,8488,7863,8571
55399,Gm21742_chrY_90837413_90844040_(+),Gm21742,chrY,90837413,90844040,+,0,0,0,0,0,0,0,0


Last step before running DESeq2: pre-filter the genes to include only those that have an average of 5 reads per sample (so a total of 40 reads across these 8 samples).

In [61]:
df_B6_total_empty_Airn7d_filtered = df_B6_total_empty_Airn7d[df_B6_total_empty_Airn7d.loc[:, "empty1_RNAseq_B6_total" : "Airn5_7d_repB_RNAseq_B6_total"].sum(axis=1) >= 40]
df_B6_total_empty_Airn7d_filtered

,id,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Airn2_7d_repA_RNAseq_B6_total,Airn2_7d_repB_RNAseq_B6_total,Airn5_7d_repA_RNAseq_B6_total,Airn5_7d_repB_RNAseq_B6_total
2,Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,276,272,80,102
31,Mrpl15_chr1_4773206_4785739_(-),Mrpl15,chr1,4773206,4785739,-,440,311,251,257,246,261,271,250
33,Lypla1_chr1_4807788_4848410_(+),Lypla1,chr1,4807788,4848410,+,513,249,272,242,199,231,205,259
34,Gm37988_chr1_4807892_4886770_(+),Gm37988,chr1,4807892,4886770,+,369,247,211,241,212,194,223,215
35,Tcea1_chr1_4857814_4897909_(+),Tcea1,chr1,4857814,4897909,+,316,155,171,159,142,155,157,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53819,Gm15246_chrX_169320622_169368190_(+),Gm15246,chrX,169320622,169368190,+,99,72,60,72,78,80,74,101
53820,BC022960_chrX_169368189_169370003_(+),BC022960,chrX,169368189,169370003,+,8,5,6,9,6,4,9,11
53822,Mid1_chrX_169685199_170005736_(+),Mid1,chrX,169685199,170005736,+,23106,647,662,570,549,510,232,315
55397,Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,21,19,10,7


For DESeq2, will need this sata in the form of a matrix. Matrices in R can have row names and column names - will want a single rowname, so I will use the "id" column here:

In [62]:
df_B6_total_empty_Airn7d_filtered2 = df_B6_total_empty_Airn7d_filtered.set_index('id')
df_B6_total_empty_Airn7d_filtered2

,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Airn2_7d_repA_RNAseq_B6_total,Airn2_7d_repB_RNAseq_B6_total,Airn5_7d_repA_RNAseq_B6_total,Airn5_7d_repB_RNAseq_B6_total
id,,,,,,,,,,,,,
Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,276,272,80,102
Mrpl15_chr1_4773206_4785739_(-),Mrpl15,chr1,4773206,4785739,-,440,311,251,257,246,261,271,250
Lypla1_chr1_4807788_4848410_(+),Lypla1,chr1,4807788,4848410,+,513,249,272,242,199,231,205,259
Gm37988_chr1_4807892_4886770_(+),Gm37988,chr1,4807892,4886770,+,369,247,211,241,212,194,223,215
Tcea1_chr1_4857814_4897909_(+),Tcea1,chr1,4857814,4897909,+,316,155,171,159,142,155,157,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gm15246_chrX_169320622_169368190_(+),Gm15246,chrX,169320622,169368190,+,99,72,60,72,78,80,74,101
BC022960_chrX_169368189_169370003_(+),BC022960,chrX,169368189,169370003,+,8,5,6,9,6,4,9,11
Mid1_chrX_169685199_170005736_(+),Mid1,chrX,169685199,170005736,+,23106,647,662,570,549,510,232,315


In [63]:
df_B6_total_empty_Airn7d_filtered2.index.names = [None]
df_B6_total_empty_Airn7d_filtered2

,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Airn2_7d_repA_RNAseq_B6_total,Airn2_7d_repB_RNAseq_B6_total,Airn5_7d_repA_RNAseq_B6_total,Airn5_7d_repB_RNAseq_B6_total
Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,276,272,80,102
Mrpl15_chr1_4773206_4785739_(-),Mrpl15,chr1,4773206,4785739,-,440,311,251,257,246,261,271,250
Lypla1_chr1_4807788_4848410_(+),Lypla1,chr1,4807788,4848410,+,513,249,272,242,199,231,205,259
Gm37988_chr1_4807892_4886770_(+),Gm37988,chr1,4807892,4886770,+,369,247,211,241,212,194,223,215
Tcea1_chr1_4857814_4897909_(+),Tcea1,chr1,4857814,4897909,+,316,155,171,159,142,155,157,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gm15246_chrX_169320622_169368190_(+),Gm15246,chrX,169320622,169368190,+,99,72,60,72,78,80,74,101
BC022960_chrX_169368189_169370003_(+),BC022960,chrX,169368189,169370003,+,8,5,6,9,6,4,9,11
Mid1_chrX_169685199_170005736_(+),Mid1,chrX,169685199,170005736,+,23106,647,662,570,549,510,232,315
Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,21,19,10,7


In [64]:
#Now remove the ID/metadata columns
df_B6_total_empty_Airn7d_filtered3 = df_B6_total_empty_Airn7d_filtered2.iloc[:, 5:14]
df_B6_total_empty_Airn7d_filtered3

,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Airn2_7d_repA_RNAseq_B6_total,Airn2_7d_repB_RNAseq_B6_total,Airn5_7d_repA_RNAseq_B6_total,Airn5_7d_repB_RNAseq_B6_total
Xkr4_chr1_3205901_3671498_(-),313,255,374,40,276,272,80,102
Mrpl15_chr1_4773206_4785739_(-),440,311,251,257,246,261,271,250
Lypla1_chr1_4807788_4848410_(+),513,249,272,242,199,231,205,259
Gm37988_chr1_4807892_4886770_(+),369,247,211,241,212,194,223,215
Tcea1_chr1_4857814_4897909_(+),316,155,171,159,142,155,157,151
...,...,...,...,...,...,...,...,...
Gm15246_chrX_169320622_169368190_(+),99,72,60,72,78,80,74,101
BC022960_chrX_169368189_169370003_(+),8,5,6,9,6,4,9,11
Mid1_chrX_169685199_170005736_(+),23106,647,662,570,549,510,232,315
Mid1-ps1_chrY_90753057_90763485_(+),31,11,16,17,21,19,10,7


In [65]:
df_B6_total_empty_Airn7d_filtered3.to_csv('2023-04-11_RMCE_B6_reads_Airn7d_empty_for_DESeq2.csv')

Mauro: "Jackson, could you create a supplemental table of chr6 allelic expression for the Airn Jordan work and place it in the "supplemental_figures_and_tables" folder in the dropbox?"

Will subset the df for B6 and CAST "total" reads (omitting intron/exon/EJC columns) and append the df_ID_columns metadata as done above for B6_total only.

I will save two csv files: one with total B6 and cast reads for all genes, and one with total B6 and cast reads for only chr6 genes.

In [66]:
#make dataframe containing only the "cast_total" columns
df_cast_total_data = df.loc[:, df.columns.str.contains('cast_total' )]
df_cast_total_data

,Airn2_7d_repA_RNAseq_cast_total,Airn2_7d_repB_RNAseq_cast_total,Airn2_repA_RNAseq_cast_total,Airn2_repB_RNAseq_cast_total,Airn5_7d_repA_RNAseq_cast_total,Airn5_7d_repB_RNAseq_cast_total,Airn5_repA_RNAseq_cast_total,Airn5_repB_RNAseq_cast_total,empty1_RNAseq_cast_total,empty2_RNAseq_cast_total,empty3_RNAseq_cast_total,empty4_RNAseq_cast_total,Xist1_RNAseq_cast_total,Xist3_10ngmL_repA_RNAseq_cast_total,Xist3_10ngmL_repB_RNAseq_cast_total,Xist3_RNAseq_cast_total,Xist4_RNAseq_cast_total,Xist5_10ngmL_repA_RNAseq_cast_total,Xist5_10ngmL_repB_RNAseq_cast_total,Xist5_RNAseq_cast_total
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,200,240,225,184,98,106,147,148,77,161,267,24,129,91,222,137,185,68,235,157
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,0,1,1,2,0,0,4,0,2,1,1,0,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55396,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55397,12,12,11,4,6,5,3,1,19,1,6,8,5,1,9,5,6,1,12,3
55398,415,416,386,339,406,416,440,364,345,361,313,279,247,208,394,332,352,190,412,331
55399,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [67]:
#Now combine df_ID_columns, df_B6_total_data, and df_cast_total_data

df_B6_cast_total = pd.concat([df_ID_columns, df_B6_total_data, df_cast_total_data], axis=1)
df_B6_cast_total

,id,gene,chr,start,end,strand,Airn2_7d_repA_RNAseq_B6_total,Airn2_7d_repB_RNAseq_B6_total,Airn2_repA_RNAseq_B6_total,Airn2_repB_RNAseq_B6_total,Airn5_7d_repA_RNAseq_B6_total,Airn5_7d_repB_RNAseq_B6_total,Airn5_repA_RNAseq_B6_total,Airn5_repB_RNAseq_B6_total,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Xist1_RNAseq_B6_total,Xist3_10ngmL_repA_RNAseq_B6_total,Xist3_10ngmL_repB_RNAseq_B6_total,Xist3_RNAseq_B6_total,Xist4_RNAseq_B6_total,Xist5_10ngmL_repA_RNAseq_B6_total,Xist5_10ngmL_repB_RNAseq_B6_total,Xist5_RNAseq_B6_total,Airn2_7d_repA_RNAseq_cast_total,Airn2_7d_repB_RNAseq_cast_total,Airn2_repA_RNAseq_cast_total,Airn2_repB_RNAseq_cast_total,Airn5_7d_repA_RNAseq_cast_total,Airn5_7d_repB_RNAseq_cast_total,Airn5_repA_RNAseq_cast_total,Airn5_repB_RNAseq_cast_total,empty1_RNAseq_cast_total,empty2_RNAseq_cast_total,empty3_RNAseq_cast_total,empty4_RNAseq_cast_total,Xist1_RNAseq_cast_total,Xist3_10ngmL_repA_RNAseq_cast_total,Xist3_10ngmL_repB_RNAseq_cast_total,Xist3_RNAseq_cast_total,Xist4_RNAseq_cast_total,Xist5_10ngmL_repA_RNAseq_cast_total,Xist5_10ngmL_repB_RNAseq_cast_total,Xist5_RNAseq_cast_total
0,4933401J01Rik_chr1_3073253_3074322_(+),4933401J01Rik,chr1,3073253,3074322,+,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Gm26206_chr1_3102016_3102125_(+),Gm26206,chr1,3102016,3102125,+,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,276,272,298,268,80,102,107,116,313,255,374,40,197,158,400,235,250,126,338,262,200,240,225,184,98,106,147,148,77,161,267,24,129,91,222,137,185,68,235,157
3,Gm18956_chr1_3252757_3253236_(+),Gm18956,chr1,3252757,3253236,+,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Gm37180_chr1_3365731_3368549_(-),Gm37180,chr1,3365731,3368549,-,1,1,2,1,0,2,0,0,2,1,0,0,3,2,1,2,0,3,1,1,0,0,1,1,0,1,1,2,0,0,4,0,2,1,1,0,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55396,Gm21860_chrY_90752427_90755467_(-),Gm21860,chrY,90752427,90755467,-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55397,Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,21,19,16,27,10,7,7,9,31,11,16,17,15,11,15,10,12,3,15,15,12,12,11,4,6,5,3,1,19,1,6,8,5,1,9,5,6,1,12,3
55398,Gm47283_chrY_90784738_90816465_(+),Gm47283,chrY,90784738,90816465,+,8277,8488,8400,7370,7863,8571,9659,7967,12349,9059,7618,8334,5441,5525,8687,8327,8477,6020,11111,9342,415,416,386,339,406,416,440,364,345,361,313,279,247,208,394,332,352,190,412,331
55399,Gm21742_chrY_90837413_90844040_(+),Gm21742,chrY,90837413,90844040,+,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
#Rearrange the columns to make them more user-friendly (group by genotype/condition)

df_B6_cast_total_grouped = df_B6_cast_total[['id', 'gene', 'chr', 'start', 'end', 'strand',                                                'empty1_RNAseq_B6_total', 'empty2_RNAseq_B6_total', 'empty3_RNAseq_B6_total', 'empty4_RNAseq_B6_total',
                                               'Airn2_repA_RNAseq_B6_total', 'Airn2_repB_RNAseq_B6_total', 'Airn5_repA_RNAseq_B6_total', 'Airn5_repB_RNAseq_B6_total',
                                               'Airn2_7d_repA_RNAseq_B6_total', 'Airn2_7d_repB_RNAseq_B6_total', 'Airn5_7d_repA_RNAseq_B6_total', 'Airn5_7d_repB_RNAseq_B6_total',
                                               'Xist3_10ngmL_repA_RNAseq_B6_total', 'Xist3_10ngmL_repB_RNAseq_B6_total', 'Xist5_10ngmL_repA_RNAseq_B6_total', 'Xist5_10ngmL_repB_RNAseq_B6_total',
                                               'Xist1_RNAseq_B6_total', 'Xist3_RNAseq_B6_total', 'Xist4_RNAseq_B6_total', 'Xist5_RNAseq_B6_total',
                                               'empty1_RNAseq_cast_total', 'empty2_RNAseq_cast_total', 'empty3_RNAseq_cast_total', 'empty4_RNAseq_cast_total',
                                               'Airn2_repA_RNAseq_cast_total', 'Airn2_repB_RNAseq_cast_total', 'Airn5_repA_RNAseq_cast_total', 'Airn5_repB_RNAseq_cast_total',
                                               'Airn2_7d_repA_RNAseq_cast_total', 'Airn2_7d_repB_RNAseq_cast_total', 'Airn5_7d_repA_RNAseq_cast_total', 'Airn5_7d_repB_RNAseq_cast_total',
                                               'Xist3_10ngmL_repA_RNAseq_cast_total', 'Xist3_10ngmL_repB_RNAseq_cast_total', 'Xist5_10ngmL_repA_RNAseq_cast_total', 'Xist5_10ngmL_repB_RNAseq_cast_total',
                                               'Xist1_RNAseq_cast_total', 'Xist3_RNAseq_cast_total', 'Xist4_RNAseq_cast_total', 'Xist5_RNAseq_cast_total']]
df_B6_cast_total_grouped

,id,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Airn2_repA_RNAseq_B6_total,Airn2_repB_RNAseq_B6_total,Airn5_repA_RNAseq_B6_total,Airn5_repB_RNAseq_B6_total,Airn2_7d_repA_RNAseq_B6_total,Airn2_7d_repB_RNAseq_B6_total,Airn5_7d_repA_RNAseq_B6_total,Airn5_7d_repB_RNAseq_B6_total,Xist3_10ngmL_repA_RNAseq_B6_total,Xist3_10ngmL_repB_RNAseq_B6_total,Xist5_10ngmL_repA_RNAseq_B6_total,Xist5_10ngmL_repB_RNAseq_B6_total,Xist1_RNAseq_B6_total,Xist3_RNAseq_B6_total,Xist4_RNAseq_B6_total,Xist5_RNAseq_B6_total,empty1_RNAseq_cast_total,empty2_RNAseq_cast_total,empty3_RNAseq_cast_total,empty4_RNAseq_cast_total,Airn2_repA_RNAseq_cast_total,Airn2_repB_RNAseq_cast_total,Airn5_repA_RNAseq_cast_total,Airn5_repB_RNAseq_cast_total,Airn2_7d_repA_RNAseq_cast_total,Airn2_7d_repB_RNAseq_cast_total,Airn5_7d_repA_RNAseq_cast_total,Airn5_7d_repB_RNAseq_cast_total,Xist3_10ngmL_repA_RNAseq_cast_total,Xist3_10ngmL_repB_RNAseq_cast_total,Xist5_10ngmL_repA_RNAseq_cast_total,Xist5_10ngmL_repB_RNAseq_cast_total,Xist1_RNAseq_cast_total,Xist3_RNAseq_cast_total,Xist4_RNAseq_cast_total,Xist5_RNAseq_cast_total
0,4933401J01Rik_chr1_3073253_3074322_(+),4933401J01Rik,chr1,3073253,3074322,+,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Gm26206_chr1_3102016_3102125_(+),Gm26206,chr1,3102016,3102125,+,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Xkr4_chr1_3205901_3671498_(-),Xkr4,chr1,3205901,3671498,-,313,255,374,40,298,268,107,116,276,272,80,102,158,400,126,338,197,235,250,262,77,161,267,24,225,184,147,148,200,240,98,106,91,222,68,235,129,137,185,157
3,Gm18956_chr1_3252757_3253236_(+),Gm18956,chr1,3252757,3253236,+,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Gm37180_chr1_3365731_3368549_(-),Gm37180,chr1,3365731,3368549,-,2,1,0,0,2,1,0,0,1,1,0,2,2,1,3,1,3,2,0,1,0,0,4,0,1,1,1,2,0,0,0,1,1,1,0,1,2,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55396,Gm21860_chrY_90752427_90755467_(-),Gm21860,chrY,90752427,90755467,-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55397,Mid1-ps1_chrY_90753057_90763485_(+),Mid1-ps1,chrY,90753057,90763485,+,31,11,16,17,16,27,7,9,21,19,10,7,11,15,3,15,15,10,12,15,19,1,6,8,11,4,3,1,12,12,6,5,1,9,1,12,5,5,6,3
55398,Gm47283_chrY_90784738_90816465_(+),Gm47283,chrY,90784738,90816465,+,12349,9059,7618,8334,8400,7370,9659,7967,8277,8488,7863,8571,5525,8687,6020,11111,5441,8327,8477,9342,345,361,313,279,386,339,440,364,415,416,406,416,208,394,190,412,247,332,352,331
55399,Gm21742_chrY_90837413_90844040_(+),Gm21742,chrY,90837413,90844040,+,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [70]:
df_B6_cast_total_grouped.to_csv('RNA-seq_allelic_reads_all_genes.csv')

In [71]:
df_B6_cast_total_grouped_chr6 = df_B6_cast_total_grouped[df_B6_cast_total_grouped['chr'] == 'chr6']
df_B6_cast_total_grouped_chr6

,id,gene,chr,start,end,strand,empty1_RNAseq_B6_total,empty2_RNAseq_B6_total,empty3_RNAseq_B6_total,empty4_RNAseq_B6_total,Airn2_repA_RNAseq_B6_total,Airn2_repB_RNAseq_B6_total,Airn5_repA_RNAseq_B6_total,Airn5_repB_RNAseq_B6_total,Airn2_7d_repA_RNAseq_B6_total,Airn2_7d_repB_RNAseq_B6_total,Airn5_7d_repA_RNAseq_B6_total,Airn5_7d_repB_RNAseq_B6_total,Xist3_10ngmL_repA_RNAseq_B6_total,Xist3_10ngmL_repB_RNAseq_B6_total,Xist5_10ngmL_repA_RNAseq_B6_total,Xist5_10ngmL_repB_RNAseq_B6_total,Xist1_RNAseq_B6_total,Xist3_RNAseq_B6_total,Xist4_RNAseq_B6_total,Xist5_RNAseq_B6_total,empty1_RNAseq_cast_total,empty2_RNAseq_cast_total,empty3_RNAseq_cast_total,empty4_RNAseq_cast_total,Airn2_repA_RNAseq_cast_total,Airn2_repB_RNAseq_cast_total,Airn5_repA_RNAseq_cast_total,Airn5_repB_RNAseq_cast_total,Airn2_7d_repA_RNAseq_cast_total,Airn2_7d_repB_RNAseq_cast_total,Airn5_7d_repA_RNAseq_cast_total,Airn5_7d_repB_RNAseq_cast_total,Xist3_10ngmL_repA_RNAseq_cast_total,Xist3_10ngmL_repB_RNAseq_cast_total,Xist5_10ngmL_repA_RNAseq_cast_total,Xist5_10ngmL_repB_RNAseq_cast_total,Xist1_RNAseq_cast_total,Xist3_RNAseq_cast_total,Xist4_RNAseq_cast_total,Xist5_RNAseq_cast_total
37942,Gm23551_chr6_3051289_3051409_(-),Gm23551,chr6,3051289,3051409,-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37943,Vmn1r-ps5_chr6_3072137_3073067_(-),Vmn1r-ps5,chr6,3072137,3073067,-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37944,Vmn1r-ps6_chr6_3107477_3112864_(-),Vmn1r-ps6,chr6,3107477,3112864,-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37945,Vmn1r261_chr6_3143627_3144554_(-),Vmn1r261,chr6,3143627,3144554,-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37946,Gm42642_chr6_3182630_3182984_(-),Gm42642,chr6,3182630,3182984,-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41066,Bicd1_chr6_149408886_149563329_(+),Bicd1,chr6,149408886,149563329,+,169,145,142,142,196,141,180,136,188,182,181,163,69,134,70,167,55,71,59,82,67,52,52,53,74,68,80,43,78,71,64,46,33,41,30,48,63,52,47,44
41067,Gm15785_chr6_149433780_149434473_(-),Gm15785,chr6,149433780,149434473,-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,1,0,2,0,0,1,1,0,2,0,0,1
41068,Gm21814_chr6_149566090_149583656_(+),Gm21814,chr6,149566090,149583656,+,19,19,18,22,26,16,26,22,24,21,19,23,11,24,7,26,15,6,10,10,2,0,5,1,2,3,2,0,0,3,0,1,1,0,1,3,5,1,1,2
41069,Gm44043_chr6_149570409_149573661_(+),Gm44043,chr6,149570409,149573661,+,4,0,5,3,8,4,11,2,8,6,3,1,2,8,2,5,2,1,2,5,2,0,0,1,2,0,0,1,2,0,0,2,0,2,0,1,0,1,0,0


In [72]:
df_B6_cast_total_grouped_chr6.to_csv('RNA-seq_allelic_reads_chr6_genes.csv')